In [58]:
from __future__ import absolute_import, division, print_function, unicode_literals

import collections
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

from sklearn.metrics import roc_auc_score, roc_curve, auc

import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
# import collections

import os
import subprocess
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

from sklearn.metrics import roc_auc_score, roc_curve, auc


def getData(meta, X_raw, tgt, future):
	"""
	Let us say that target 'tgt' is the index of a tile, 
	so we predict for all three variables, CASE
	"""
	
	#AR = meta.loc[tgt, 'CASE']
	AR = meta.loc[tgt, 'BURGLARY-THEFT-MOTOR_VEHICLE_THEFT']
	PR = meta.loc[tgt, 'HOMICIDE-ASSAULT-BATTERY']
	
	#print('Arrest: {},\nProperty: {},\nAssault: {}.'.format(AR, PR, AS))
	
	X = X_raw[[PR]].T
	Y = X_raw[[AR]].T

	# get train and test
	# Train is the first two years (2014, 2015), so of length 365 * 2 = 730
	# test is the last year (2016, a leap year), so of length 366
	X_train = X[:30 - future]
	Y_train = Y[future: 30]

	X_test = X[30 - future: -future]
	Y_test = Y[30:]
	
	# Reshape to fit RNN
	# The dimension of RNN input/output is (num_samples, length_in_time, data_dimension)
	X_train = X_train.reshape(1, *X_train.shape)
	Y_train = Y_train.reshape(1, *Y_train.shape)

	X_test = X_test.reshape(1, *X_test.shape)
	Y_test = Y_test.reshape(1, *Y_test.shape)
	
	print('Training data: input dim = {}, output dim = {}'.format(X_train.shape, Y_train.shape))
	print('Out-sample data: input dim = {}, output dim = {}'.format(X_test.shape, Y_test.shape))

	return X_train, Y_train, X_test, Y_test


def train(X_train, Y_train, epochs=200):

	model = tf.keras.Sequential()


	# Two LSTM layers
	model.add(LSTM(units=30, input_shape=(None, X_train.shape[-1]), return_sequences=True))
	model.add(LSTM(units=10, return_sequences=True))
	
	# One output layers
	model.add(TimeDistributed(Dense(units=1, activation='sigmoid')))
	
	model.compile(loss='mse', optimizer='adam')

	model.summary()

	model.fit(X_train, Y_train, epochs=epochs, batch_size=1, verbose=1)
	
	return model


def train_3(X_train, Y_train, epochs=200):

	model = tf.keras.Sequential()

	# Three LSTM layers
	model.add(LSTM(units=50, input_shape=(None, X_train.shape[-1]), return_sequences=True))
	model.add(LSTM(units=10, return_sequences=True))
	model.add(LSTM(units=10, return_sequences=True))
	
	# One output layers
	model.add(TimeDistributed(Dense(units=1, activation='sigmoid')))
	
	model.compile(loss='mse', optimizer='adam')

	model.summary()

	model.fit(X_train, Y_train, epochs=epochs, batch_size=1, verbose=1)
	
	return model



In [59]:
meta = pd.read_csv('meta.csv', index_col=0)
#meta = pd.read_csv('meta2.csv', index_col=0)
X_raw = np.genfromtxt('CRIME-_2014-01-01_2016-12-31.csv')
#X_raw = np.genfromtxt('../time_series_covid/data.csv')
X_raw
#meta.columns=['CASE','lat1','lat2','lon1','lon2']

array([[0., 0., 0., ..., 0., 1., 0.],
       [2., 0., 1., ..., 3., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [60]:
cynetTop = meta.index.values

In [61]:
X_raw.shape

(6165, 1096)

## An example

In [62]:
center = cynetTop[0]
tile = meta.loc[center, ['lat1', 'lat2','lon1', 'lon2']]
    
future = 10
    
X_train, Y_train, X_test, Y_test = getData(meta, X_raw, center, future)

model = train_3(X_train, Y_train, epochs=100)
prediction = model.predict(X_test)

Training data: input dim = (1, 20, 1), output dim = (1, 20, 1)
Out-sample data: input dim = (1, 1066, 1), output dim = (1, 1066, 1)
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, None, 50)          10400     
_________________________________________________________________
lstm_13 (LSTM)               (None, None, 10)          2440      
_________________________________________________________________
lstm_14 (LSTM)               (None, None, 10)          840       
_________________________________________________________________
time_distributed_6 (TimeDist (None, None, 1)           11        
Total params: 13,691
Trainable params: 13,691
Non-trainable params: 0
_________________________________________________________________
Train on 1 samples
Epoch 1/100
1/1 [==============================] - 3s 3s/sample - loss: 0.2500
Epoch 2/100
1/1 [=

Epoch 89/100
1/1 [==============================] - 0s 29ms/sample - loss: 0.0900
Epoch 90/100
1/1 [==============================] - 0s 32ms/sample - loss: 0.0898
Epoch 91/100
1/1 [==============================] - 0s 31ms/sample - loss: 0.0897
Epoch 92/100
1/1 [==============================] - 0s 32ms/sample - loss: 0.0896
Epoch 93/100
1/1 [==============================] - 0s 30ms/sample - loss: 0.0895
Epoch 94/100
1/1 [==============================] - 0s 31ms/sample - loss: 0.0893
Epoch 95/100
1/1 [==============================] - 0s 30ms/sample - loss: 0.0892
Epoch 96/100
1/1 [==============================] - 0s 31ms/sample - loss: 0.0891
Epoch 97/100
1/1 [==============================] - 0s 31ms/sample - loss: 0.0890
Epoch 98/100
1/1 [==============================] - 0s 30ms/sample - loss: 0.0888
Epoch 99/100
1/1 [==============================] - 0s 31ms/sample - loss: 0.0887
Epoch 100/100
1/1 [==============================] - 0s 36ms/sample - loss: 0.0886


In [63]:
figTitle = '#'.join(map(str, tile.values)) + '_{}'.format(future)
dfName = figTitle + '.rnnres'

In [64]:
Y_test=np.squeeze(Y_test)
prediction=np.squeeze(prediction)

In [65]:
from sklearn.metrics import roc_auc_score
np.corrcoef(Y_test, prediction)

array([[1.        , 0.12131079],
       [0.12131079, 1.        ]])

In [66]:
Y_test

array([0., 0., 2., ..., 0., 0., 0.])

In [67]:
prediction

array([0.46160716, 0.41556615, 0.33804274, ..., 0.10946661, 0.10946659,
       0.10946657], dtype=float32)